In [29]:
import pandas as pd
import chess
from tensorflow.keras.models import load_model
import numpy as np


2024-04-17 21:08:32.610633: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [30]:
def castling_rights_to_integer(fen_castling):
    rights = 0
    if 'K' in fen_castling:
        rights |= 0b1000
    if 'Q' in fen_castling:
        rights |= 0b0100
    if 'k' in fen_castling:
        rights |= 0b0010
    if 'q' in fen_castling:
        rights |= 0b0001
    return rights

def en_passant_to_integer(fen_en_passant):
    if fen_en_passant == '-':
        return -1
    
    rn_passant = 0
    
    if 'a' in fen_en_passant:
        rn_passant != 0b0000
    elif 'b' in fen_en_passant:
        rn_passant |= 0b0001
    elif 'c' in fen_en_passant:
        rn_passant |= 0b0010
    elif 'd' in fen_en_passant:
        rn_passant |= 0b0011
    elif 'e' in fen_en_passant:
        rn_passant |= 0b0100
    elif 'f' in fen_en_passant:
        rn_passant |= 0b0101
    elif 'g' in fen_en_passant:
        rn_passant |= 0b0110
    elif 'h' in fen_en_passant:
        rn_passant |= 0b0111
    
    if '3' in fen_en_passant:
        rn_passant |= 0b1000
    elif '6' in fen_en_passant:
        rn_passant |= 0b0000
        
    return rn_passant

def fen_to_encoded(fen):
    piece_mapping = {
    'p': 1, 'P': -1,  # pawn
    'n': 2, 'N': -2,  # knight
    'b': 3, 'B': -3,  # bishop
    'r': 4, 'R': -4,  # rook
    'q': 5, 'Q': -5,  # queen
    'k': 6, 'K': -6,  # king
    '.': 0  # empty square
    
    }
    board_state, turn, castling, en_passant, halfmove_clock, fullmove_number = fen.split()

    encoded_board = np.full((8, 8), 0, dtype=int)
    rank_index = 0
    file_index = 0

    for char in board_state:
        if char == '/':
            rank_index += 1
            file_index = 0
        elif char.isdigit():
            file_index += int(char)
        else:
            piece_index = piece_mapping[char]
            encoded_board[rank_index, file_index] = piece_index
            file_index += 1

    encoded_board = encoded_board.reshape((1,(encoded_board.shape[0]*encoded_board.shape [1])))
    encoded_board = np.append(encoded_board, 1 if turn == "w" else -1)
    encoded_board = np.append(encoded_board, castling_rights_to_integer(castling))
    encoded_board = np.append(encoded_board, en_passant_to_integer(en_passant))
    encoded_board = np.append(encoded_board, np.array([halfmove_clock,fullmove_number]))
    return encoded_board.astype(int)
    

In [31]:
model = load_model('models/linear_mlp.h5')

X_kaufman = pd.read_csv('kaufman.csv')

X_kaufman = fen_to_encoded(X_kaufman)

2024-04-17 21:08:38.934216: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 8, 8, 12], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_4'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']

In [ ]:
predictions = model.predict(X_kaufman)

# Step 4: Post-process Predictions (if needed)
# Depending on your problem, you may need to convert the predictions to a certain format or interpret the results.

# Example: Convert predictions to class labels (assuming it's a classification problem)
predicted_labels = predictions.argmax(axis=-1)

# You can then use 'predicted_labels' for further analysis or evaluation.


In [ ]:
data = pd.read_csv('kaufman.csv')
fen = data['FEN']
board = chess.Board()
for f in fen:
    f = f.split(" ")
    b = f[0]
    turn = f[1]
    
    board.set_board_fen(b)
    if turn == 'w':
        board.turn = chess.WHITE
    elif turn == "b":
        board.turn = chess.BLACK
    else:
        print("error")
    for move in board.legal_moves:
        print(move)